In [1]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import scipy.linalg.interpolative as sli

import cmath
import math
%matplotlib inline
import matplotlib.pyplot as plt
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
bempp.api.show_available_platforms_and_devices()
bempp.api.set_default_device(0, 0)

0: Portable Computing Language
    0: pthread-Intel(R) Xeon(R) W-2155 CPU @ 3.30GHz


In [2]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*wn*r))*scipy.special.jv(q+0.5, wn*r)

def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [3]:
d = [1,0,0] # wave's travel direction
wn = 5 # wavenumber
eta = wn
deg = 16
h = 0.1

In [4]:
# far field points: unit sphere
unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_sphere = unit_sphere.vertices
space_sphere = bempp.api.function_space(unit_sphere, 'P', 1)

# scatterer: cube
dom_cube = bempp.api.shapes.cube(length = 2/1.8, origin=(-1/1.8,-1/1.8,-1/1.8), h = h)
vert_cube = dom_cube.vertices
space_cube = bempp.api.function_space(dom_cube,'P',1)

identity = bempp.api.operators.boundary.sparse.identity(space_cube, space_cube,space_cube)
slp = bempp.api.operators.boundary.helmholtz.single_layer(space_cube,space_cube,space_cube,wn)
dlp = bempp.api.operators.boundary.helmholtz.double_layer(space_cube,space_cube,space_cube,wn)
slp_far = helmholtz_farfield.single_layer(space_cube, vert_sphere, wn)
dlp_far = helmholtz_farfield.double_layer(space_cube, vert_sphere,wn)

lhs = 0.5 * identity + dlp - 1j* eta *slp

mass_mat = bempp.api.operators.boundary.sparse.identity(space_sphere,space_sphere,space_sphere).weak_form().A

In [5]:
slp_far_field_coeff = []
sph_harm_coeff = []

for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def dirichlet_fun(x,n,domain_index,result):
                with objmode():
                    result[0] = - regular_spherical_wavefunctions(p,q,x)

        rhs_fun = bempp.api.GridFunction(space_cube, fun = dirichlet_fun)
        field, info = bempp.api.linalg.gmres(lhs, rhs_fun, tol=1E-10)
        slp_far_field_coeff.append(dlp_far.evaluate(field) - 1j * eta * slp_far.evaluate(field))

        @bempp.api.complex_callable
        def sph_harm(x,n,domain_index,result):
            with objmode():
                result[0] = normalized_spherical_harmonics(p,q,x)
        sph_harm_grid = bempp.api.GridFunction(space_sphere,fun = sph_harm)
        sph_harm_coeff.append(np.conj(sph_harm_grid.coefficients)/((-1j)**(q+1)))

In [6]:
T_matrix = np.zeros(((deg+1)**2,(deg+1)**2),dtype = complex)
for i in range((deg+1)**2):
    for j in range((deg+1)**2):
        T_matrix[i,j] = (slp_far_field_coeff[j] @ mass_mat @ sph_harm_coeff[i])*wn

In [7]:
mat1 = T_matrix + np.identity(T_matrix.shape[0])

In [8]:
u,s,vh = np.linalg.svd(T_matrix)
u1,s1,vh1 = np.linalg.svd(T_matrix + np.identity(T_matrix.shape[0]))

In [10]:
s

array([9.84913179e-01, 9.84885301e-01, 9.84813769e-01, 8.55766676e-01,
       8.55758237e-01, 7.85548192e-01, 7.85535850e-01, 7.85531985e-01,
       7.34786595e-01, 5.47090942e-01, 5.47072439e-01, 5.47010073e-01,
       5.09260891e-01, 4.92617386e-01, 4.92609490e-01, 4.92570439e-01,
       2.60476656e-01, 2.60413396e-01, 2.60399827e-01, 1.66726516e-01,
       1.66708283e-01, 1.32626757e-01, 1.32625736e-01, 1.32616010e-01,
       5.98293542e-02, 5.98158175e-02, 5.98101829e-02, 4.70104932e-02,
       4.57008427e-02, 4.56997623e-02, 3.73512073e-02, 3.73469194e-02,
       3.73454364e-02, 2.26367358e-02, 2.26352894e-02, 2.26340221e-02,
       7.41943984e-03, 7.41881725e-03, 7.41757835e-03, 6.37958431e-03,
       6.37796745e-03, 6.37609786e-03, 4.90958083e-03, 3.12437971e-03,
       3.12391253e-03, 3.12344013e-03, 2.88684389e-03, 2.88671392e-03,
       1.91399806e-03, 9.49820422e-04, 6.80405178e-04, 6.80340550e-04,
       6.80280765e-04, 5.18503029e-04, 5.18461049e-04, 5.18412658e-04,
      

In [12]:
n = T_matrix.shape[0]
nboxes = int(np.sqrt(n))
size = n // nboxes

In [13]:
def get_box(nx, ny, A, nboxes, n):
    """Return the content of a given box"""
    xrange = [nx * size, (1 + nx) * size] 
    yrange = [ny * size, (1 + ny) * size] 
    return A[xrange[0]:xrange[1], yrange[0]:yrange[1]] 

In [14]:
def get_con_box(i,A,nboxes,n):
    """Return Ai"""
    BB = A[i * size : (i + 1) * size, :]
    B = np.delete(BB, np.s_[i*size : (i+1)*size], axis = 1)
    
    CC = np.transpose(A[:, i * size : (i + 1) * size])
    C = np.delete(CC, np.s_[i*size : (i+1)*size], axis = 1)
    return np.hstack((B,C))

In [15]:
def L(i,k,A,nboxes,n):
    idx1,proj1 = sli.interp_decomp(get_con_box(i,A,nboxes,n), k, rand = False)
    B1 = sli.reconstruct_skel_matrix(get_con_box(i,A,nboxes,n),k,idx1)
    B1_trans = np.matrix.getH(B1)
    
    idx2,proj2 = sli.interp_decomp(B1_trans, k, rand = False)
    P2_trans = sli.reconstruct_interp_matrix(idx2,proj2)

    P2 = np.matrix.getH(P2_trans)
    return P2,idx2

In [16]:
def index_set(k,A,nboxes,n):
    l = []
    for i in range(nboxes):
        l.append(L(i,k,A,nboxes,n)[1][0:k])
    return l

In [17]:
def SSS(k,A,nboxes,n):
    ll = index_set(k,A,nboxes,n)
    mat = np.zeros((k*nboxes,k*nboxes),dtype = complex)
    for i in range(nboxes):
        for j in range(nboxes):
            if i != j:
                mat_ =  get_box(i,j,A,nboxes,n)[ll[i],:]
                mat[i*k:(i+1)*k,j*k:(j+1)*k] = mat_[:,ll[j]]
    return mat

In [18]:
def R(i,k,A,nboxes,n):
    RR = np.matrix.getH(L(i,k,A,nboxes,n)[0])
    return RR

In [19]:
def new_S(k,A,nboxes,n):
    mat_ = SSS(k,A,nboxes,n)
    for i in range(nboxes):
        mat_[i*k:(i+1)*k,i*k:(i+1)*k] = np.linalg.inv(R(i,k,A,nboxes,n) @ np.linalg.inv(get_box(i,i,A,nboxes,n)) @ L(i,k,A,nboxes,n)[0])
    return mat_

In [20]:
def logdet_method_complex(k,A,nboxes,n):
    a1 = 0
    p1,l1,u1 = scipy.linalg.lu(new_S(k,A,nboxes,n))
    for i in range(k*nboxes):
        a1 += cmath.log(u1[i,i])
        
    a2 = 0
    for i in range (nboxes):
        p2,l2,u2 = scipy.linalg.lu(get_box(i,i,A,nboxes,n))
        for j in range(size):
            a2 +=cmath.log(u2[j,j])
            
    a3 = 0
    for i in range(nboxes):
        p3,l3,u3 = scipy.linalg.lu(-R(i,k,A,nboxes,n)@np.linalg.inv(get_box(i,i,A,nboxes,n)) @L(i,k,A,nboxes,n)[0] )
        for j in range(k):
            a3 += cmath.log(u3[j,j])
    temp =  a1 + a2 + a3 - cmath.log((-1)**(k*nboxes))
    return np.real(temp) + cmath.log(np.exp(np.imag(temp)*1j))

In [21]:
logdet_method_complex(size, T_matrix, nboxes,n)

LinAlgError: Singular matrix

In [22]:
P_exact1,L_exact1,U_exact1 = scipy.linalg.lu(T_matrix)
logdet_exact1 = 0
for i in range(T_matrix.shape[0]):
    print( U_exact1[i,i], i)
    
#logdet_exact = np.real(logdet_exact1) + cmath.log(np.exp(np.imag(logdet_exact1)*1j))
#print(logdet_exact)

(-0.1727762750465436-0.3034061254758609j) 0
(-0.5951234237551216+0.43222780323726606j) 1
(-0.5951251650530603+0.43221900239896255j) 2
(-0.5951430801144563+0.43221950895094136j) 3
(-0.8406114418921714-0.28406403586176204j) 4
(-0.961665442008479-0.13831214503874736j) 5
(-0.7195536476049609-0.4298512663963768j) 6
(-0.9616671371570132-0.1382709520360494j) 7
(-0.8603751801279288-0.31939169791049804j) 8
(-0.2789213618067706-0.4400219228229286j) 9
(-0.4225497570588381-0.4736447900878935j) 10
(-0.2875811810322965-0.44581114760091534j) 11
(-0.2659101600204743-0.4313916997465064j) 12
(-0.28684742751411113-0.4456503873278809j) 13
(-0.40184033297315574-0.4857801810412907j) 14
(-0.27814745327634316-0.4399012848067872j) 15
(-0.01812585205346911-0.12597651451289943j) 16
(-0.06360014314284144-0.2378049719897826j) 17
(-0.04975262725434099-0.21040860136595704j) 18
(-0.02409599746066326-0.14536357498491378j) 19
(-0.01076906886684946-0.09230134200407877j) 20
(-0.01996669888993026-0.13853941159616842j) 21


In [27]:
U_exact1[224,224]

(-8.598508932308571e-18-1.238835704150822e-15j)

In [23]:
logdet_method_complex(size,mat1,nboxes,n)

(-10.418602047515533+2.31187382836964j)

In [24]:
P_exact2,L_exact2,U_exact2 = scipy.linalg.lu(mat1)
logdet_exact2 = 0
for i in range(T_matrix.shape[0]):
    logdet_exact2 += cmath.log(U_exact2[i,i])
    
logdet_exact22 = np.real(logdet_exact2) + cmath.log(np.exp(np.imag(logdet_exact2)*1j))
print(logdet_exact22)

(-10.418602047515556+2.3118738283700258j)


In [25]:
np.linalg.inv(mat1)

array([[ 1.00111659e+00+4.05993173e-01j, -1.21208980e-06-4.17602544e-06j,
        -1.75733417e-06-1.83843510e-06j, ...,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 1.89530002e-05-1.19480079e-05j,  1.00585436e+00-1.17102028e+00j,
         3.10134662e-05+7.97085185e-06j, ...,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 1.08715113e-05-1.18048064e-05j,  2.61831284e-06-3.78946360e-05j,
         1.00588481e+00-1.17100607e+00j, ...,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       ...,
       [-8.37250553e-06-9.48031944e-09j,  4.07988875e-05+4.31539025e-06j,
         2.03309534e-06+9.85737592e-07j, ...,
         1.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 1.74976050e-05-4.79800806e-06j, -6.